# Black Scholes Exercise 2: NumPy implementation

- Use numpy
- Use cProfile and VTune to look for bottlenecks and hotspots in the code

In [1]:
# Boilerplate for the example

import cProfile
import pstats
import numpy as np

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = np.zeros(nopt, dtype=np.float64)
put  = -np.ones(nopt, dtype=np.float64)

# The NumPy modified naive Black Scholes algorithm (looped)

- Minimally converted code from the Naive example
- TODO: Convert the math import to numpy variants

In [8]:
import numpy as np
# TODO we need numpy's log, invsqrt, exp, erf
from numpy import log, invsqrt, exp, erf
def black_scholes(nopt, price, strike, t, rate, vol, call, put):
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    for i in range(nopt):
        P = price[i]
        S = strike[i]
        T = t[i]
        
        a = log(P / S)
        b = T * mr
        
        z = T * sig_sig_two
        c = 0.25 * z
        y = invsqrt(z)
        
        w1 = (a - b + c) * y
        w2 = (a - b - c) * y
        
        d1 = 0.5 + 0.5 * erf(w1)
        d2 = 0.5 + 0.5 * erf(w2)
        
        Se = exp(b) * S
        
        call[i] = P * d1 - Se * d2
        put[i] = call[i] - P + Se
        #print(call,put)

## Run timeit and/or cProfile to see what is happening

In [11]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

647 ms ± 9.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%load_ext line_profiler
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Timer unit: 1e-06 s

Total time: 1.56723 s
File: <ipython-input-8-170e66250949>
Function: black_scholes at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def black_scholes(nopt, price, strike, t, rate, vol, call, put):
     5         1         10.0     10.0      0.0      mr = -rate
     6         1          8.0      8.0      0.0      sig_sig_two = vol * vol * 2
     7                                               
     8    100001      53544.0      0.5      3.4      for i in range(nopt):
     9    100000      59358.0      0.6      3.8          P = price[i]
    10    100000      55930.0      0.6      3.6          S = strike[i]
    11    100000      55832.0      0.6      3.6          T = t[i]
    12                                                   
    13    100000     184005.0      1.8     11.7          a = log(P / S)
    14    100000      60414.0      0.6      3.9          b = T * mr
    15                       

## _Why does this example take longer, even with performance libraries?_

When using NumPy, we need to unloop the function and utilize NumPy Arrays to achieve vectorization.

Remove the loop and let it work on whole arrays!

In [12]:
from numpy import log, invsqrt, exp, erf

def black_scholes(nopt, price, strike, t, rate, vol):
    mr = -rate
    sig_sig_two = vol * vol * 2
    P = price
    S = strike
    T = t

    a = log(P / S)
    b = T * mr

    z = T * sig_sig_two
    c = 0.25 * z
    y = invsqrt(z)

    w1 = (a - b + c) * y
    w2 = (a - b - c) * y

    d1 = 0.5 + 0.5 * erf(w1)
    d2 = 0.5 + 0.5 * erf(w2)

    Se = exp(b) * S

    call = P * d1 - Se * d2
    put = call - P + Se

    return call, put

## Run timeit and/or cProfile to see what is happening

In [13]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

3.98 ms ± 43.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

Timer unit: 1e-06 s

Total time: 0.017789 s
File: <ipython-input-12-878269c4691b>
Function: black_scholes at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def black_scholes(nopt, price, strike, t, rate, vol):
     4         1         19.0     19.0      0.1      mr = -rate
     5         1         12.0     12.0      0.1      sig_sig_two = vol * vol * 2
     6         1          6.0      6.0      0.0      P = price
     7         1          5.0      5.0      0.0      S = strike
     8         1          5.0      5.0      0.0      T = t
     9                                           
    10         1       5994.0   5994.0     33.7      a = log(P / S)
    11         1       1068.0   1068.0      6.0      b = T * mr
    12                                           
    13         1       1160.0   1160.0      6.5      z = T * sig_sig_two
    14         1       1022.0   1022.0      5.7      c = 0.25 * z
    15         

## What does VTune show us?
You can use the provided alias from amplxe.ipy to run VTune/amplifier.

In [16]:
%pycat amplxe.ipy

%alias amplxe amplxe-cl -c hotspots -target-duration-type=veryshort -q -r hs$$ -- python ./%l && amplxe-gui hs$$


In [17]:
%run amplxe.ipy

In [18]:
# save code (select the right cells!)
%save -f runme.py 1 2 4
# collect data
%amplxe runme.py

The following commands were written to file `runme.py`:
# Boilerplate for the example

import cProfile
import pstats
import numpy as np

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = np.zeros(nopt, dtype=np.float64)
put  = -np.ones(nopt, dtype=np.float64)
import numpy as np
# TODO we need numpy's log, invsqrt, exp, erf

def black_scholes(nopt, price, strike, t, rate, vol, call, put):
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    for i in range(nopt):
        P = price[i]
 